# Clean everything

In [ ]:
import os
import shutil

current_folder = os.getcwd()

for item in os.listdir(current_folder):
    item_path = os.path.join(current_folder, item)
    
    if os.path.isfile(item_path) or os.path.islink(item_path):
        os.remove(item_path)
    elif os.path.isdir(item_path):
        shutil.rmtree(item_path)

print("All files and folders have been removed!")


All files and folders have been removed!
All files and folders have been removed!


# Download data

In [2]:
!curl -L -o {curr_path}/dataset.zip https://www.kaggle.com/competitions/digit-recognizer

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Failed to create the file {curr_path}/dataset.zip: No such file or 
100  1369    0  1369    0     0   5020      0 --:--:-- --:--:-- --:--:--  5033
curl: (23) Failure writing output to destination


# Library

In [3]:
import os
import shutil
import pandas as pd
import numpy as np
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [4]:
curr_path = os.getcwd()

In [5]:
# os.makedirs(f"{curr_path}/dataset", exist_ok = True)
# if "dataset.zip" in os.listdir(f"{curr_path}/"):
#     shutil.move(f"{curr_path}/dataset.zip", f"{curr_path}/dataset/dataset.zip")
# temp = f"{curr_path}/dataset/dataset.zip"
# des = f"{curr_path}/dataset/"
# if not os.path.exists(temp):
#     print("Error: ZIP file not found!")
# with zipfile.ZipFile("/kaggle/working/dataset/dataset.zip", 'r') as zip_ref:
#     zip_ref.extractall(des)

In [6]:
input_path = "/kaggle/input/digit-recognizer"

In [7]:
train_df = pd.read_csv(f"{input_path}/train.csv")
test_df = pd.read_csv(f"{input_path}/test.csv")

In [8]:
train_df.describe()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [9]:
train_df['label'].unique()

array([1, 0, 4, 7, 3, 5, 8, 9, 2, 6])

# Train model

## Model 1

In [10]:
# # Preprocess data
# X = train_df.iloc[:, 1:].values / 255.0  # Normalize pixel values
# y = train_df.iloc[:, 0].values           # Labels

# # Train-test split
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Convert to PyTorch tensors
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.long)
# X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
# y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# # Create DataLoaders
# batch_size = 64
# train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)

# # Define the MLP model using nn.Sequential
# class MLPModel(nn.Module):
#     def __init__(self, input_size=784, hidden_size=256, output_size=10):
#         super(MLPModel, self).__init__()
        
#         self.model = nn.Sequential(
#             nn.Linear(input_size, hidden_size),
#             nn.ReLU(),
#             nn.Dropout(0.3),

#             nn.Linear(hidden_size, hidden_size),
#             nn.ReLU(),
#             nn.Dropout(0.3),

#             nn.Linear(hidden_size, output_size)
#         )

#     def forward(self, x):
#         return self.model(x)

# # Initialize model, loss function, and optimizer
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = MLPModel().to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# model_path = "a1.pth"

# # Training loop
# num_epochs = 300
# best_loss = float('inf')
# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
    
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
        
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()

#     curr_loss = running_loss/len(train_loader)
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {curr_loss:.4f}")

#     # Check if validation loss improved
#     if curr_loss < best_loss:
#         best_loss = curr_loss
#         torch.save(model.state_dict(), model_path)
#         print(f"✅ Epoch {epoch+1}: Validation loss improved to {curr_loss:.4f}. Model saved!")
#         epochs_no_improve = 0
#     else:
#         epochs_no_improve += 1
#         print(f"⚠️ No improvement in validation loss for {epochs_no_improve} epochs.")

# # Validation
# model.eval()
# correct, total = 0, 0
# with torch.no_grad():
#     for inputs, labels in val_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         outputs = model(inputs)
#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print(f"Validation Accuracy: {100 * correct / total:.2f}%")

## Model 2

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
# Preprocess data
X = train_df.iloc[:, 1:].values / 255.0  # Normalize pixel values
y = train_df.iloc[:, 0].values           # Labels

# Train-validation split (80-20)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)

# Define the MLP model using nn.Sequential
class MLPModel(nn.Module):
    def __init__(self, input_size=784, hidden_size=[512,256,128,64], output_size=10):
        super(MLPModel, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size[0]),
            nn.BatchNorm1d(hidden_size[0]),
            # nn.ReLU(),
            nn.SiLU(),
            nn.Dropout(0.3),
           
            nn.Linear(hidden_size[0], hidden_size[0]),
            nn.BatchNorm1d(hidden_size[0]),
            # nn.ReLU(),
            # nn.LeakyReLU(negative_slope=0.03),
            nn.Tanh(),
            # nn.Sigmoid(),
            nn.Dropout(0.3),

            nn.Linear(hidden_size[0], hidden_size[0]),
            nn.BatchNorm1d(hidden_size[0]),
            # nn.ReLU(),
            nn.LeakyReLU(negative_slope=0.03),
            # nn.Tanh(),
            # nn.Sigmoid(),
            nn.Dropout(0.4),
            
            nn.Linear(hidden_size[0], hidden_size[0]),
            nn.BatchNorm1d(hidden_size[0]),
            nn.ReLU(),
            # nn.Mish(),
            # nn.LeakyReLU(negative_slope=0.03),
            # nn.Tanh(),
            # nn.Sigmoid(),
            nn.Dropout(0.4),
            
            nn.Linear(hidden_size[0], output_size)
        )

    def forward(self, x):
        return self.model(x)

# Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPModel().to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# optimizer = optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-4)
optimizer = optim.RAdam(model.parameters(), lr=0.001, betas=(0.9, 0.9999), weight_decay=1e-4)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4, nesterov=True)



scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.8, patience=5)
# scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
# scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=num_epochs)

# Training loop with best model saving
num_epochs = 300
patience = 200  # Number of epochs to wait before stopping
best_val_loss = float("inf")  # Start with a very high loss
best_val_acc = 0.0
model_path = "best_mlp_digit_recognizer.pth"
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total

    current_lr = optimizer.param_groups[0]['lr']
    scheduler.step(avg_val_loss)
    new_lr = optimizer.param_groups[0]['lr']
    if new_lr < current_lr:
        print(f"📘 Learning rate reduced from {current_lr:.6f} to {new_lr:.6f}")
    
    # Check if validation loss improved
    if val_accuracy > best_val_acc:
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy 
        torch.save(model.state_dict(), model_path)
        print(f"✅ Epoch {epoch+1}: Validation loss improved to {best_val_loss:.4f}. Model saved!")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"⚠️ No improvement in validation loss for {epochs_no_improve} epochs.")

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

    # Stop training if no improvement for 'patience' epochs
    if epochs_no_improve >= patience:
        print("🚨 Early stopping triggered! Loading best model...")
        break

# Load the best model
model.load_state_dict(torch.load(model_path))
print(f"🎯 Best model loaded with Val Loss: {best_val_loss:.4f}")

Using device: cuda
✅ Epoch 1: Validation loss improved to 0.7135. Model saved!
Epoch [1/300], Train Loss: 1.1828, Val Loss: 0.7135, Val Acc: 93.39%
✅ Epoch 2: Validation loss improved to 0.6575. Model saved!
Epoch [2/300], Train Loss: 0.7458, Val Loss: 0.6575, Val Acc: 95.27%
✅ Epoch 3: Validation loss improved to 0.6258. Model saved!
Epoch [3/300], Train Loss: 0.6845, Val Loss: 0.6258, Val Acc: 96.18%
✅ Epoch 4: Validation loss improved to 0.6035. Model saved!
Epoch [4/300], Train Loss: 0.6554, Val Loss: 0.6035, Val Acc: 96.74%
✅ Epoch 5: Validation loss improved to 0.5897. Model saved!
Epoch [5/300], Train Loss: 0.6335, Val Loss: 0.5897, Val Acc: 97.18%
⚠️ No improvement in validation loss for 1 epochs.
Epoch [6/300], Train Loss: 0.6181, Val Loss: 0.5874, Val Acc: 97.15%
✅ Epoch 7: Validation loss improved to 0.5818. Model saved!
Epoch [7/300], Train Loss: 0.6062, Val Loss: 0.5818, Val Acc: 97.20%
✅ Epoch 8: Validation loss improved to 0.5709. Model saved!
Epoch [8/300], Train Loss: 

<ipython-input-12-21994229cdcc>:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


## Model 2 with skip connection

In [ ]:
# Define the Residual Block with Skip Connection
class ResidualBlock(nn.Module):
    def __init__(self, in_features, out_features, activation=nn.SiLU(), dropout_rate=0.3):
        """
        in_features (int): Number of input features.
        out_features (int): Number of output features.
        activation (nn.Module): Activation function to use (default: SiLU).
        dropout_rate (float): Dropout rate (default: 0.3).
        """
        super(ResidualBlock, self).__init__()
        
        # Main layers
        self.linear = nn.Linear(in_features, out_features)
        self.bn = nn.BatchNorm1d(out_features)
        self.activation = activation
        self.dropout = nn.Dropout(dropout_rate)
        
        # Skip connection: If input and output dimensions don't match, use a linear layer to adjust dimensions
        self.skip = nn.Linear(in_features, out_features) if in_features != out_features else None

    def forward(self, x):
        identity = x  # Save the input for the skip connection
        out = self.linear(x)
        out = self.bn(out)
        out = self.activation(out)
        out = self.dropout(out)
        
        # Apply skip connection
        if self.skip is not None:
            identity = self.skip(identity)
        out += identity  # Add the skip connection
        return out

# Define the MLP Model with Residual Blocks
class MLPModel(nn.Module):
    def __init__(self, input_size=784, hidden_size=[512, 256, 128, 64], output_size=10):
        super(MLPModel, self).__init__()
        self.layers = nn.ModuleList([
            # ResidualBlock(input_size, hidden_size[0],nn.SiLU(),0.3),  # First residual block
            # ResidualBlock(hidden_size[0], hidden_size[1], nn.LeakyReLU(negative_slope=0.03), 0.3),  # Second residual block
            # ResidualBlock(hidden_size[1], hidden_size[2], nn.Tanh(), 0.2),  # Third residual block
            # ResidualBlock(hidden_size[2], hidden_size[3], nn.Mish(), 0.2),  # Fourth residual block
            # nn.Linear(hidden_size[3], output_size)  # Final output layer

            
            nn.Linear(input_size, hidden_size[0]),
            nn.BatchNorm1d(hidden_size[0]),
            # nn.ReLU(),
            nn.SiLU(),
            nn.Dropout(0.3),
           
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.BatchNorm1d(hidden_size[1]),
            # nn.ReLU(),
            # nn.LeakyReLU(negative_slope=0.03),
            nn.Tanh(),
            # nn.Sigmoid(),
            nn.Dropout(0.3),

            ResidualBlock(hidden_size[1], hidden_size[2], nn.LeakyReLU(negative_slope=0.03),0.2)  # Third residual block          

            nn.Linear(hidden_size[2], hidden_size[3]),
            nn.BatchNorm1d(hidden_size[3]),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(hidden_size[3], output_size)
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Preprocess data (example)
X = train_df.iloc[:, 1:].values / 255.0  # Normalize pixel values
y = train_df.iloc[:, 0].values  # Labels

# Train-validation split (80-20)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)

# Initialize model, loss function, and optimizer
model = MLPModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RAdam(model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.8, patience=10)

# Training loop with best model saving
num_epochs = 300
patience = 200  # Number of epochs to wait before stopping
best_val_loss = float("inf")  # Start with a very high loss
best_val_acc = 0.0
model_path = "best_mlp_digit_recognizer.pth"
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total

    current_lr = optimizer.param_groups[0]['lr']
    scheduler.step(val_accuracy)
    new_lr = optimizer.param_groups[0]['lr']
    if new_lr < current_lr:
        print(f"📘 Learning rate reduced from {current_lr:.6f} to {new_lr:.6f}")
    
    # Check if validation loss improved
    if val_accuracy > best_val_acc:
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy 
        torch.save(model.state_dict(), model_path)
        print(f"✅ Epoch {epoch+1}: Validation loss improved to {best_val_loss:.4f}. Model saved!")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"⚠️ No improvement in validation loss for {epochs_no_improve} epochs.")

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

    # Stop training if no improvement for 'patience' epochs
    if epochs_no_improve >= patience:
        print("🚨 Early stopping triggered! Loading best model...")
        break

# Load the best model
model.load_state_dict(torch.load(model_path))
print(f"🎯 Best model loaded with Val Loss: {best_val_loss:.4f}")

## Nháp

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
# Preprocess data
X = train_df.iloc[:, 1:].values / 255.0  # Normalize pixel values
y = train_df.iloc[:, 0].values           # Labels

# Train-validation split (80-20)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)

# Define the MLP model using nn.Sequential
class MLPModel(nn.Module):
    def __init__(self, input_size=784, hidden_size=[512,256,128,64], output_size=10):
        super(MLPModel, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size[0]),
            nn.BatchNorm1d(hidden_size[0]),
            # nn.ReLU(),
            # nn.SiLU(),
            nn.Tanh(),
            nn.Dropout(0.3),
           
            nn.Linear(hidden_size[0], hidden_size[0]),
            nn.BatchNorm1d(hidden_size[0]),
            # nn.ReLU(),
            # nn.LeakyReLU(negative_slope=0.03),
            # nn.Tanh(),
            nn.Sigmoid(),
            nn.Dropout(0.3),

            nn.Linear(hidden_size[0], hidden_size[0]),
            nn.BatchNorm1d(hidden_size[0]),
            # nn.ReLU(),
            # nn.LeakyReLU(negative_slope=0.03),
            nn.Tanh(),
            # nn.Sigmoid(),
            nn.Dropout(0.4),
            
            nn.Linear(hidden_size[0], hidden_size[0]),
            nn.BatchNorm1d(hidden_size[0]),
            # nn.ReLU(),
            # nn.Mish(),
            # nn.LeakyReLU(negative_slope=0.03),
            # nn.Tanh(),
            nn.Sigmoid(),
            nn.Dropout(0.4),
            
            nn.Linear(hidden_size[0], output_size)
        )

    def forward(self, x):
        return self.model(x)

# Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPModel().to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# optimizer = optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-4)
optimizer = optim.RAdam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), weight_decay=1e-4)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4, nesterov=True)



scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5)
# scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
# scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=num_epochs)

# Training loop with best model saving
num_epochs = 300
patience = 200  # Number of epochs to wait before stopping
best_val_loss = float("inf")  # Start with a very high loss
best_val_acc = 0.0
model_path = "best_mlp_digit_recognizer.pth"
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total

    current_lr = optimizer.param_groups[0]['lr']
    scheduler.step(avg_val_loss)
    new_lr = optimizer.param_groups[0]['lr']
    if new_lr < current_lr:
        print(f"📘 Learning rate reduced from {current_lr:.6f} to {new_lr:.6f}")
    
    # Check if validation loss improved
    if val_accuracy > best_val_acc:
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy 
        torch.save(model.state_dict(), model_path)
        print(f"✅ Epoch {epoch+1}: Validation loss improved to {best_val_loss:.4f}. Model saved!")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"⚠️ No improvement in validation loss for {epochs_no_improve} epochs.")

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

    # Stop training if no improvement for 'patience' epochs
    if epochs_no_improve >= patience:
        print("🚨 Early stopping triggered! Loading best model...")
        break

# Load the best model
model.load_state_dict(torch.load(model_path))
print(f"🎯 Best model loaded with Val Loss: {best_val_loss:.4f}")

# Evaluation

In [ ]:
torch.save(model.state_dict(), "best_mlp_digit_recognizer.pth")
# torch.save(model.state_dict(), "mlp_digit_recognizer.pth")
# torch.save(model.state_dict(), "a1.pth")
print("Model saved successfully!")

In [ ]:
model = MLPModel().to(device)  # Recreate the model structure
# Load the best model for final evaluation
model.load_state_dict(torch.load(model_path))
model.eval()


# =======================
# Validate the model
# =======================
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

# =======================
# Test the model on test.csv
# =======================
X_test = test_df.values / 255.0  # Normalize test images
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

with torch.no_grad():
    test_outputs = model(X_test_tensor)
    predictions = torch.argmax(test_outputs, dim=1).cpu().numpy()  # Convert to numpy array

# Save predictions to submission.csv (for Kaggle)
submission = pd.DataFrame({"ImageId": np.arange(1, len(predictions) + 1), "Label": predictions})
submission.to_csv("submission.csv", index=False)
print("Predictions saved to submission.csv!")